Автор материала: Зраев Артем.

Можно использовать в каких угодно целях.

<b> В задании нужно загрузить датасет с данными оттока и ответить на несколько вопросов (написать код). При этом сам датасет уже есть и его необязательно качать с репозитория</b>

Цель задания: проверить базовые навыки работы студентов с Pandas, умение проводить такой же базовый EDA (exploratory data analysis), делать feature engineering и обучать и валидировать модель.

Список столбцов с типами данных в датасете:

- customerID           object
- gender               object
- SeniorCitizen         int64
- Partner              object
- Dependents           object
- tenure                int64
- PhoneService         object
- MultipleLines        object
- InternetService      object
- OnlineSecurity       object
- OnlineBackup         object
- DeviceProtection     object
- TechSupport          object
- StreamingTV          object
- StreamingMovies      object
- Contract             object
- PaperlessBilling     object
- PaymentMethod        object
- MonthlyCharges      float64
- TotalCharges         object
- Churn                object

Данные лежат на google диске по [ссылке](https://drive.google.com/file/d/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ/view?usp=sharing).

In [1]:
!wget 'https://drive.google.com/uc?export=download&id=1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ' -O WA_Fn-UseC_-Telco-Customer-Churn.csv

--2022-05-14 22:44:16--  https://drive.google.com/uc?export=download&id=1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ
Resolving drive.google.com (drive.google.com)... 142.251.107.139, 142.251.107.101, 142.251.107.113, ...
Connecting to drive.google.com (drive.google.com)|142.251.107.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-08-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i2l3bpcjbvu5goucuvpt9nrmk42pasop/1652568225000/14904333240138417226/*/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ?e=download [following]
--2022-05-14 22:44:17--  https://doc-08-c0-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/i2l3bpcjbvu5goucuvpt9nrmk42pasop/1652568225000/14904333240138417226/*/1g8lsZO25O2SxuCrIYVSDV3eLqmfC-TCZ?e=download
Resolving doc-08-c0-docs.googleusercontent.com (doc-08-c0-docs.googleusercontent.com)... 142.251.107.132, 2607:f8b0:400c:c32::84
Connecting to doc-08-c0-docs.googleusercontent.com (doc-08-c0

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv("./WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head(3)

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes


##### 1. Какое соотношение мужчин и женщин в представленном наборе данных?

In [3]:
df['gender'].value_counts(normalize=True)

Male      0.504756
Female    0.495244
Name: gender, dtype: float64

##### 2. Какое количество уникальных значений у поля InternetService?

In [4]:
df['InternetService'].nunique()

3

##### 3. Выведите статистики по полю TotalCharges (median, mean, std).

In [5]:
df['TotalCharges'].describe()

count     7043
unique    6531
top           
freq        11
Name: TotalCharges, dtype: object

В чем странность того, что вы получили? (подсказка: смотреть нужно на тип данных)

Потому что строковый тип данных, пандас думает, что это категориальный признак

##### 4. Сделайте замену значений поля PhoneService  на числовые (Yes->1, No->0)

In [6]:
df['PhoneService'] = df['PhoneService'].map({'Yes': 1, 'No': 0})

##### 5. Сделайте замену пробелов в поле TotalCharges на np.nan и приведите поле к типу данных float32. Затем заполните оставшиеся пропуски значением 0 с помощью метода fillna у столбца. Снова выведите статистики и сравните с тем, что вы видели в вопросе 3

In [7]:
df['TotalCharges'] = df['TotalCharges'].replace(' ', np.nan).astype(np.float32).fillna(0)
df['TotalCharges'].describe()

count    7043.000000
mean     2279.732178
std      2266.794434
min         0.000000
25%       398.549988
50%      1394.550049
75%      3786.599976
max      8684.799805
Name: TotalCharges, dtype: float64

##### 6. Сделайте замену значений поля Churn на числовые (Yes -> 1, No - 0)

In [8]:
df['Churn'] = df['Churn'].map({'Yes': 1, 'No': 0})  # factorize()

##### 7. Сделайте замену значений полей StreamingMovies, StreamingTV, TechSupport  на числовые (Yes -> 1, No -> 0, No internet service->0)

In [9]:
for col in ('StreamingMovies', 'StreamingTV', 'TechSupport'):
    df[col] = df[col].map({'Yes': 1, 'No': 0, 'No internet service': 0})

##### 8. Заполните пропуски в поле PhoneService значением 0

In [10]:
df['PhoneService'].fillna(0, inplace=True)

##### 8. Для нашего датасета оставьте только указанный ниже список полей, удалив все другие и выведите верхние 3 строки

In [11]:
columns = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 
           'StreamingMovies', 'StreamingTV', 'TechSupport', 'Churn']

df = df[columns]

##### 9. Разделите датасет на тренировочную и тестовую выборку.

In [12]:
features = ['gender', 'tenure', 'PhoneService', 'TotalCharges', 'StreamingMovies', 'StreamingTV', 'TechSupport']
target = 'Churn'

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.3, shuffle=True, stratify=df[target], random_state=42)

##### 10. соберите pipeline для поля gender (нужно разобраться и изучить https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html ) из классов ColumnSelector и OHEEncoder, которые уже написаны ниже заранее

In [14]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline

class ColumnSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[self.key]
    
class NumberSelector(BaseEstimator, TransformerMixin):
    """
    Transformer to select a single column from the data frame to perform additional transformations on
    Use on numeric columns in the data
    """
    def __init__(self, key):
        self.key = key

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        return X[[self.key]]
    
class OHEEncoder(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key
        self.columns = []

    def fit(self, X, y=None):
        self.columns = [col for col in pd.get_dummies(X, prefix=self.key).columns]
        return self

    def transform(self, X):
        X = pd.get_dummies(X, prefix=self.key)
        test_columns = [col for col in X.columns]
        for col_ in test_columns:
            if col_ not in self.columns:
                X[col_] = 0
        return X[self.columns]

gender = Pipeline(steps=[('selecetor', ColumnSelector(key='gender')),
                         ('encoder', OHEEncoder(key='gender'))])

##### 11. Вызовите метод fit_transform у пайплайна gender и передайте туда нашу тренировочную выборку (пример по ссылке из документации https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline.fit)

In [15]:
gender.fit_transform(X_train)

,gender_Female,gender_Male
5557,1,0
2270,1,0
6930,1,0
2257,1,0
898,1,0
...,...,...
4250,0,1
1488,0,1
6303,1,0
2710,1,0


##### 12. Здесь код писать уже не нужно (все сделано за вас). К полю tenure применяем StandardScaler (нормируем и центрируем). Ссылка - https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html
Вопрос - в каких случаях это может быть полезно?

In [16]:
from sklearn.preprocessing import StandardScaler

tenure =  Pipeline([
                ('selector', NumberSelector(key='tenure')),
                ('standard', StandardScaler())
            ])

Полезно для приведения вещественных данных к одному масштабу, тем самым не придавая слишком большое или маленькое значение данному признаку, при использовании в линейных моделях (использующих методы оптимизации) или моделях, основанных на вычислении расстояния между образцами для их разделения на классы. 

##### 13. Напишите аналогичный (как для tenure) преобразователь поля TotalCharges

In [30]:
TotalCharges = Pipeline(steps=[('selector', NumberSelector(key='TotalCharges')),
                                ('scaler', StandardScaler())])

Объединение всех "кубиков" очень легко сделать таким образом

In [18]:
from sklearn.pipeline import FeatureUnion

number_features = Pipeline([
                ('selector', ColumnSelector(key=['PhoneService',
                                                 'StreamingMovies', 'StreamingTV', 
                                                 'TechSupport']))
            ])

In [31]:
feats = FeatureUnion([('tenure', tenure),
                      ('TotalCharges', TotalCharges),
                      ('continuos_features', number_features),
                      ('gender', gender)])
feature_processing = Pipeline([('feats', feats)])

На этом этапе что мы сделали:
1. написали преобразователь поля gender, который делает OHE кодирование
2. написали преобразователь для поля tenure, который нормирует и центрирует его 
3. повторили п. 2 для поля TotalCharges
3. для всех остальных просто взяли признаки как они есть, без изменений

У нас уже готов наш пайплайн, который преобразовывает признаки. Давайте обучим модель поверх него. В качестве модели возьмем RandomForestClassifier

In [36]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('features', feature_processing),
    ('classifier', RandomForestClassifier(n_estimators=200, max_depth=5, class_weight='balanced', random_state=42)),
], verbose=True).fit(X_train, y_train)

[Pipeline] .......... (step 1 of 2) Processing features, total=   0.0s
[Pipeline] ........ (step 2 of 2) Processing classifier, total=   1.2s


##### 14. Сделайте прогноз вероятности оттока для X_test с помощью нашего предобученного на предыдущем шаге пайплайна и убедитесь что вам возвращаются вероятности для 2 классов

In [37]:
test_probas = pipeline.predict_proba(X_test)
test_probas

array([[0.28758529, 0.71241471],
       [0.47793913, 0.52206087],
       [0.28666227, 0.71333773],
       ...,
       [0.65614786, 0.34385214],
       [0.31939298, 0.68060702],
       [0.76132031, 0.23867969]])

##### 15. Посчитайте метрики качества получившейся модели (roc_auc)

In [38]:
from sklearn.metrics import roc_auc_score, classification_report

print(classification_report(y_test, test_probas[:,1] >= 0.5, digits=3))
print(f'ROC-AUC score: {roc_auc_score(y_test, test_probas[:,1]):.3f}')

              precision    recall  f1-score   support

           0      0.875     0.711     0.785      1552
           1      0.474     0.718     0.571       561

    accuracy                          0.713      2113
   macro avg      0.674     0.715     0.678      2113
weighted avg      0.768     0.713     0.728      2113

ROC-AUC score: 0.796


In [41]:
from sklearn.metrics import precision_recall_curve

precision, recall, thresholds = precision_recall_curve(y_test, test_probas[:,1])
fscore = 2 * (precision * recall) / (precision + recall)
idx = np.argmax(fscore)
print('Best Threshold=%f, F-Score=%.3f, Precision=%.3f, Recall=%.3f' % (thresholds[idx], fscore[idx], precision[idx], recall[idx]))

Best Threshold=0.445026, F-Score=0.578, Precision=0.447, Recall=0.816


### Сохраним наш пайплайн

In [39]:
import dill
with open("model_RF.dill", "wb") as f:
    dill.dump(pipeline, f)

That's it!